# 하이브리드 기반 추천 시스템

In [2]:
from pymongo import MongoClient
import numpy as np
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
uri = "mongodb://localhost:27017/"
music_db = "music"
music_collection = "music"
target_user = "최민호"

In [4]:
def load_music_data(uri, db_name, collection_name):
    print("1. 음악 데이터를 불러옵니다...")
    client = MongoClient(uri)
    db = client[db_name]
    collection = db[collection_name]
    data = list(collection.find({}, {"_id": 0}))
    print(f"불러온 곡 개수: {len(data)}")
    print(f"예시 곡 데이터: {data[0] if data else '없음'}")
    return data

In [5]:
def make_string(text):
    return " ".join(text) if isinstance(text, list) else str(text)

def make_genres_list(genres):
    if isinstance(genres, list):
        return [g.strip().lower() for g in genres]
    elif isinstance(genres, str):
        return [genres.strip().lower()]
    else:
        return ["unknown"]

In [6]:
def train_word2vec(music_data, vector_size=100):
    print("2. Word2Vec 모델을 학습합니다...")
    sentences = []
    for song in music_data:
        lyrics = make_string(song.get("lyrics", "")).lower()
        words = lyrics.split()
        if words:
            sentences.append(words)

    if not sentences:
        raise ValueError('가사 문장이 없음')

    model = Word2Vec(sentences, vector_size=vector_size, window=5, min_count=1, workers=4, epochs=5)
    print(f"Word2Vec 단어 수: {len(model.wv.key_to_index)}")
    return model

In [7]:
def get_lyrics_vector(lyrics, model):
    words = make_string(lyrics).lower().split()
    word_vectors = [model.wv[word] for word in words if word in model.wv]
    return np.mean(word_vectors, axis=0) if word_vectors else np.zeros(model.vector_size)

def get_genre_vector(genres, all_genres):
    onehot = np.zeros(len(all_genres))
    genres_list = make_genres_list(genres)
    for g in genres_list:
        for i, genre_name in enumerate(all_genres):
            if g == genre_name:
                onehot[i] = 1
    return onehot

def make_song_vectors(music_data, model, all_genres):
    print("3. 곡 벡터를 생성합니다...")
    vectors = {}
    for song in music_data:
        title = song.get("title", "")
        if not title:
            continue
        lyrics_vec = get_lyrics_vector(song.get("lyrics", ""), model)
        genre_vec = get_genre_vector(song.get("genres", ""), all_genres)
        combined_vec = np.concatenate([lyrics_vec, genre_vec])
        vectors[title] = combined_vec
    print(f"생성된 곡 벡터 수: {len(vectors)}")
    return vectors

In [8]:
def load_user_item_matrix(uri, db_name="user"):
    print("4. 유저-아이템 행렬을 생성합니다...")
    client = MongoClient(uri)
    db = client[db_name]
    user_ids = db.list_collection_names()

    all_titles = set()
    user_likes = {}

    for user_id in user_ids:
        collection = db[user_id]
        titles = [doc["title"] for doc in collection.find({}, {"_id": 0}) if "title" in doc]
        user_likes[user_id] = titles
        all_titles.update(titles)

    all_titles = sorted(all_titles)
    title_index = {title: i for i, title in enumerate(all_titles)}

    matrix = np.zeros((len(user_ids), len(all_titles)))
    for i, user_id in enumerate(user_ids):
        for title in user_likes[user_id]:
            if title in title_index:
                matrix[i][title_index[title]] = 1

    print(f"유저 수: {len(user_ids)}, 곡 수: {len(all_titles)}")
    return user_ids, all_titles, matrix, user_likes

In [9]:
def hybrid_recommendation(target_user, music_data, user_ids, matrix, user_likes, song_vectors, model_vector_size, all_genres, top_k=3, top_n=5):
    print(f"5. '{target_user}'에 대한 하이브리드 추천을 수행합니다...")
    if target_user not in user_ids:
        raise ValueError("해당 유저 없음")

    user_idx = user_ids.index(target_user)
    target_vec = matrix[user_idx].reshape(1, -1)

    similarities = []
    for i, uid in enumerate(user_ids):
        if uid == target_user:
            continue
        sim = cosine_similarity(target_vec, matrix[i].reshape(1, -1))[0][0]
        similarities.append((uid, sim))

    similarities.sort(key=lambda x: x[1], reverse=True)
    top_users = similarities[:top_k]
    print(f"유사한 사용자 Top-{top_k}: {[uid for uid, _ in top_users]}")

    target_likes = set(user_likes[target_user])
    print(f"'{target_user}'가 좋아한 곡 수: {len(target_likes)}")

    cf_candidates = {}
    for uid, sim in top_users:
        for song in user_likes[uid]:
            if song not in target_likes:
                cf_candidates[song] = max(cf_candidates.get(song, 0), sim)

    print(f"후보 곡 수 (CF 기반): {len(cf_candidates)}")

    if not target_likes:
        raise ValueError("타겟 유저가 좋아한 곡이 없습니다.")

    # 기준 곡 수동 선택 (가장 첫 번째 곡 사용)
    for song in target_likes:
        if song in song_vectors:
            ref_vec = song_vectors[song]
            break
    else:
        raise ValueError("기준이 될 곡 벡터가 없습니다.")

    final_scores = []
    for song, cf_score in cf_candidates.items():
        cbf_vec = song_vectors.get(song)
        if cbf_vec is None:
            continue
        cbf_score = cosine_similarity(ref_vec.reshape(1, -1), cbf_vec.reshape(1, -1))[0][0]
        hybrid_score = 0.6 * cf_score + 0.4 * cbf_score
        final_scores.append((song, hybrid_score))

    final_scores.sort(key=lambda x: x[1], reverse=True)
    print(f"최종 추천 곡 수: {len(final_scores[:top_n])}")
    return final_scores[:top_n]


In [10]:
music_data = load_music_data(uri, music_db, music_collection)
model = train_word2vec(music_data)

all_genres_set = set()
for song in music_data:
    genres = make_genres_list(song.get("genres", ""))
    all_genres_set.update(genres)
all_genres = sorted(list(all_genres_set))
print(f"장르 리스트: {all_genres}")

song_vectors = make_song_vectors(music_data, model, all_genres)
user_ids, all_titles, matrix, user_likes = load_user_item_matrix(uri)

recommendations = hybrid_recommendation(
    target_user, music_data, user_ids, matrix,
    user_likes, song_vectors, model.vector_size,
    all_genres, top_k=3, top_n=5
)

print(f"\n'{target_user}'에게 추천할 곡:")
for title, score in recommendations:
    print(f"- {title} (점수: {score:.4f})")


1. 음악 데이터를 불러옵니다...
불러온 곡 개수: 173
예시 곡 데이터: {'spotify_id': '6xUgc0eCkUrtQikLhbLaB0', 'title': '청춘만화', 'release_date': '2022-05-26', 'duration': '3:46', 'artist': '이무진', 'genres': ['k-pop', 'k-indie'], 'lyrics': []}
2. Word2Vec 모델을 학습합니다...
Word2Vec 단어 수: 5029
장르 리스트: ['alternative', 'alternative rock', 'art rock', 'baroque pop', 'britpop', 'dance', 'dance, k-pop', 'dance-pop', 'disco', 'edm', 'electronic', 'funk', 'funk rock', 'grunge', 'hard rock', 'hip-hop', 'house', 'indie, modern rock', 'j-pop', 'j-rock', 'k-ballad', 'k-hip-hop', 'k-hiphop', 'k-indie', 'k-pop', 'k-r&b', 'k-rap', 'k-rock', 'lo-fi', 'new wave', 'ost', 'pop', 'pop rock', 'power ballad', 'r&b', 'rap', 'rock', 'soundtrack', 'symphonic rock', 'synth-pop']
3. 곡 벡터를 생성합니다...
생성된 곡 벡터 수: 162
4. 유저-아이템 행렬을 생성합니다...
유저 수: 11, 곡 수: 157
5. '최민호'에 대한 하이브리드 추천을 수행합니다...
유사한 사용자 Top-3: ['김태호', '류희철', '김태현']
'최민호'가 좋아한 곡 수: 30
후보 곡 수 (CF 기반): 42
최종 추천 곡 수: 5

'최민호'에게 추천할 곡:
- 동이 틀 때 (점수: 0.4774)
- 가까운듯 먼 그대여 (점수: 0.3521)
- 사랑하지마요 (

In [1]:
from pymongo import MongoClient
import numpy as np
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity

def load_music_data(uri, db_name, collection_name):
    print("[1] 음악 데이터를 불러옵니다...")
    client = MongoClient(uri)
    db = client[db_name]
    coll = db[collection_name]
    data = list(coll.find({}, {"_id": 0}))
    print(f"   → 총 곡 수: {len(data)}")
    return data

def load_user_item_matrix(uri):
    print("\n[2] 유저 데이터를 불러옵니다...")
    client = MongoClient(uri)
    db = client["user"]
    user_ids = db.list_collection_names()
    all_titles = set()
    user_likes = {}
    for uid in user_ids:
        likes = [d["title"] for d in db[uid].find({}, {"_id": 0}) if "title" in d]
        user_likes[uid] = likes
        all_titles.update(likes)
    all_titles = sorted(list(all_titles))
    title_index = {t: i for i, t in enumerate(all_titles)}
    matrix = np.zeros((len(user_ids), len(all_titles)))
    for i, uid in enumerate(user_ids):
        for title in user_likes[uid]:
            if title in title_index:
                matrix[i][title_index[title]] = 1
    print(f"   → 유저 수: {len(user_ids)}, 곡 수: {len(all_titles)}")
    return user_ids, all_titles, matrix, user_likes

def train_word2vec(music_data):
    print("\n[3] Word2Vec 모델 학습 중...")
    sentences = []
    for song in music_data:
        lyrics = song.get("lyrics", [])
        words = lyrics if isinstance(lyrics, list) else str(lyrics).split()
        if words:
            sentences.append([w.lower() for w in words])
    model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4, epochs=5)
    print(f"   → 학습된 단어 수: {len(model.wv)}")
    return model

def make_song_vectors(music_data, model, all_genres):
    print("\n[4] 곡 벡터 생성 중...")
    vectors = {}
    for song in music_data:
        title = song.get("title", "").strip()
        if not title:
            continue
        lyrics = song.get("lyrics", [])
        words = lyrics if isinstance(lyrics, list) else str(lyrics).split()
        word_vecs = [model.wv[w.lower()] for w in words if w.lower() in model.wv]
        lyrics_vec = np.mean(word_vecs, axis=0) if word_vecs else np.zeros(model.vector_size)

        genres = song.get("genres", [])
        genre_list = [g.strip().lower() for g in genres] if isinstance(genres, list) else [str(genres).strip().lower()]
        genre_vec = np.zeros(len(all_genres))
        for g in genre_list:
            if g in all_genres:
                genre_vec[all_genres.index(g)] = 1

        combined = np.concatenate([lyrics_vec, genre_vec])
        vectors[title.lower()] = combined
    return vectors

def hybrid_recommendation(target_user, ref_song, user_ids, matrix, user_likes, song_vectors, model_vector_size, top_k=2, top_n=5):
    print("\n[5] 하이브리드 추천 수행 중...")
    idx = user_ids.index(target_user)
    target_vec = matrix[idx].reshape(1, -1)

    print(f"   → 사용자 '{target_user}'의 협업 유사도 계산:")
    sims = []
    for i, uid in enumerate(user_ids):
        if uid == target_user:
            continue
        sim = cosine_similarity(target_vec, matrix[i].reshape(1, -1))[0][0]
        sims.append((uid, sim))
        print(f"     - {uid}: {sim:.4f}")
    sims.sort(key=lambda x: x[1], reverse=True)
    top_users = [u for u, _ in sims[:top_k]]

    # 후보 곡 수집
    target_likes = set(user_likes[target_user])
    candidates = {}
    for uid in top_users:
        for song in user_likes[uid]:
            if song not in target_likes:
                candidates[song] = sims[[u for u, _ in sims].index(uid)][1]  # CF 유사도

    # CBF 기준 곡 벡터
    ref_key = ref_song.lower().strip()
    if ref_key not in song_vectors:
        raise ValueError(f"기준 곡 '{ref_song}'의 벡터가 없습니다.")
    ref_vec = song_vectors[ref_key]
    print(f"\n   → 기준 곡: '{ref_song}' (벡터 길이: {len(ref_vec)})")

    # 최종 유사도 계산
    results = []
    for song, cf_score in candidates.items():
        song_key = song.lower().strip()
        if song_key not in song_vectors:
            continue
        cb_score = cosine_similarity(
            ref_vec.reshape(1, -1),
            song_vectors[song_key].reshape(1, -1)
        )[0][0]
        hybrid_score = (0.4 * cf_score) + (0.6 * cb_score)
        artist = next((s["artist"] for s in music_data if s["title"].lower().strip() == song_key), "Unknown")
        print(f"   → '{song}' | CF: {cf_score:.4f}, CBF: {cb_score:.4f}, Hybrid: {hybrid_score:.4f}")
        results.append((song, artist, hybrid_score))
    results.sort(key=lambda x: x[2], reverse=True)
    return results[:top_n]

# 실행
if __name__ == "__main__":
    uri = "mongodb://localhost:27017/"
    music_data = load_music_data(uri, "music", "music")
    user_ids, all_titles, matrix, user_likes = load_user_item_matrix(uri)

    model = train_word2vec(music_data)
    all_genres = sorted({g.strip().lower() for song in music_data for g in (song.get("genres") or [])})
    song_vectors = make_song_vectors(music_data, model, all_genres)

    target_user = "최민호"
    ref_song = user_likes[target_user][0]
    results = hybrid_recommendation(
        target_user,
        ref_song,
        user_ids,
        matrix,
        user_likes,
        song_vectors,
        model.vector_size,
        top_k=2,
        top_n=5
    )

    print(f"\n[최종 추천 결과 for '{target_user}']:")
    for title, artist, score in results:
        print(f"- {title} | {artist} | (점수: {score:.4f})")


[1] 음악 데이터를 불러옵니다...
   → 총 곡 수: 173

[2] 유저 데이터를 불러옵니다...
   → 유저 수: 11, 곡 수: 157

[3] Word2Vec 모델 학습 중...
   → 학습된 단어 수: 2516

[4] 곡 벡터 생성 중...

[5] 하이브리드 추천 수행 중...
   → 사용자 '최민호'의 협업 유사도 계산:
     - 윤승서: 0.0333
     - 류희철: 0.1155
     - 서동진: 0.0000
     - 백나리: 0.0577
     - 규진: 0.0000
     - 김태호: 0.1291
     - 김태현: 0.0667
     - 이승은: 0.0000
     - 천지성: 0.0577
     - 김규표: 0.0000

   → 기준 곡: '청춘만화' (벡터 길이: 140)
   → '사랑하지마요' | CF: 0.1291, CBF: 0.5000, Hybrid: 0.3516
   → 'Rush (feat. 정인)' | CF: 0.1291, CBF: 0.0000, Hybrid: 0.0516
   → '동이 틀 때' | CF: 0.1291, CBF: 0.7070, Hybrid: 0.4758
   → 'TONIGHT' | CF: 0.1291, CBF: 0.7071, Hybrid: 0.4759
   → 'Cafe' | CF: 0.1291, CBF: 0.7071, Hybrid: 0.4759
   → '그래서 그래 (feat. 윤하)' | CF: 0.1291, CBF: 0.0000, Hybrid: 0.0516
   → '나는 반딧불' | CF: 0.1155, CBF: -0.0000, Hybrid: 0.0462
   → '뜨래요' | CF: 0.1155, CBF: -0.0000, Hybrid: 0.0462
   → '고독하구만' | CF: 0.1155, CBF: 0.0000, Hybrid: 0.0462
   → '봄 내음보다 너를' | CF: 0.1155, CBF: -0.0000, Hybrid: 0.0462
   

In [1]:
from pymongo import MongoClient
import numpy as np
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity

# ---------- 유틸리티 함수 ----------

def normalize_title(title):
    return str(title).strip().lower()

def process_lyrics(lyrics):
    if isinstance(lyrics, list):
        return [w.lower() for w in lyrics]
    return [w.lower() for w in str(lyrics).split()]

def get_mongo_db(uri, db_name):
    client = MongoClient(uri)
    return client[db_name]

# ---------- 데이터 불러오기 ----------

def load_music_data(db):
    print("[1] 음악 데이터를 불러옵니다...")
    data = list(db["music"].find({}, {"_id": 0}))
    print(f"   → 총 곡 수: {len(data)}")
    return data

def load_user_item_matrix(db):
    print("\n[2] 유저 데이터를 불러옵니다...")
    user_ids = db.list_collection_names()
    all_titles = set()
    user_likes = {}
    for uid in user_ids:
        likes = [d["title"] for d in db[uid].find({}, {"_id": 0}) if "title" in d]
        user_likes[uid] = likes
        all_titles.update(likes)

    all_titles = sorted(list(all_titles))
    title_index = {t: i for i, t in enumerate(all_titles)}
    uid_to_index = {uid: i for i, uid in enumerate(user_ids)}

    matrix = np.zeros((len(user_ids), len(all_titles)))
    for uid, likes in user_likes.items():
        for title in likes:
            if title in title_index:
                matrix[uid_to_index[uid]][title_index[title]] = 1

    print(f"   → 유저 수: {len(user_ids)}, 곡 수: {len(all_titles)}")
    return user_ids, uid_to_index, all_titles, matrix, user_likes

# ---------- Word2Vec ----------

def train_word2vec(music_data):
    print("\n[3] Word2Vec 모델 학습 중...")
    sentences = []
    for song in music_data:
        words = process_lyrics(song.get("lyrics", []))
        if words:
            sentences.append(words)
    model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4, epochs=5)
    print(f"   → 학습된 단어 수: {len(model.wv)}")
    return model

# ---------- 곡 벡터 생성 ----------

def make_song_vectors(music_data, model, genre_to_index):
    print("\n[4] 곡 벡터 생성 중...")
    vectors = {}
    genre_dim = len(genre_to_index)

    for song in music_data:
        title = normalize_title(song.get("title", ""))
        if not title:
            continue

        words = process_lyrics(song.get("lyrics", []))
        word_vecs = [model.wv[w] for w in words if w in model.wv]
        lyrics_vec = np.mean(word_vecs, axis=0) if word_vecs else np.zeros(model.vector_size)

        genre_vec = np.zeros(genre_dim)
        for g in song.get("genres", []):
            key = g.strip().lower()
            if key in genre_to_index:
                genre_vec[genre_to_index[key]] = 1

        combined_vec = np.concatenate([lyrics_vec, genre_vec])
        vectors[title] = combined_vec

    return vectors

# ---------- 하이브리드 추천 ----------

def hybrid_recommendation(target_user, ref_song, user_ids, uid_to_index, matrix, user_likes, song_vectors, music_data, top_k=2, top_n=5):
    print("\n[5] 하이브리드 추천 수행 중...")
    idx = uid_to_index[target_user]
    target_vec = matrix[idx].reshape(1, -1)

    print(f"   → 사용자 '{target_user}'의 협업 유사도 계산:")
    sims = []
    for uid in user_ids:
        if uid == target_user:
            continue
        other_vec = matrix[uid_to_index[uid]].reshape(1, -1)
        sim = cosine_similarity(target_vec, other_vec)[0][0]
        sims.append((uid, sim))
        print(f"     - {uid}: {sim:.4f}")

    sims.sort(key=lambda x: x[1], reverse=True)
    top_users = [u for u, _ in sims[:top_k]]
    target_likes = set(user_likes[target_user])

    candidates = {}
    for uid in top_users:
        cf_score = dict(sims)[uid]
        for song in user_likes[uid]:
            if song not in target_likes:
                candidates[song] = cf_score

    ref_key = normalize_title(ref_song)
    if ref_key not in song_vectors:
        raise ValueError(f"기준 곡 '{ref_song}'의 벡터가 없습니다.")
    ref_vec = song_vectors[ref_key]

    print(f"\n   → 기준 곡: '{ref_song}' (벡터 길이: {len(ref_vec)})")

    results = []
    for song, cf_score in candidates.items():
        song_key = normalize_title(song)
        if song_key not in song_vectors:
            continue
        cb_score = cosine_similarity(ref_vec.reshape(1, -1), song_vectors[song_key].reshape(1, -1))[0][0]
        hybrid_score = 0.4 * cf_score + 0.6 * cb_score
        artist = next((s.get("artist", "Unknown") for s in music_data if normalize_title(s["title"]) == song_key), "Unknown")
        print(f"   → '{song}' | CF: {cf_score:.4f}, CBF: {cb_score:.4f}, Hybrid: {hybrid_score:.4f}")
        results.append((song, artist, hybrid_score))

    results.sort(key=lambda x: x[2], reverse=True)
    return results[:top_n]

def evaluate_precision_recall_at_k(recommended, actual_likes, k=5):
    """
    recommended: [(title, artist, score), ...] 형식의 추천 결과
    actual_likes: 리스트 또는 집합으로 주어진 실제 좋아요 곡 제목들
    """
    recommended_titles = [title.lower().strip() for title, _, _ in recommended[:k]]
    recommended_set = set(recommended_titles)
    actual_set = set(map(str.lower, actual_likes))

    true_positives = recommended_set & actual_set
    precision = len(true_positives) / k
    recall = len(true_positives) / len(actual_set) if actual_set else 0.0

    print(f"\n[Precision@{k}]: {precision:.4f}")
    print(f"[Recall@{k}]:    {recall:.4f}")
    return precision, recall


# ---------- 메인 실행 ----------

if __name__ == "__main__":
    # MongoDB 접속
    uri = "mongodb://localhost:27017/"
    db_music = get_mongo_db(uri, "music")
    db_user = get_mongo_db(uri, "user")

    # 데이터 불러오기 및 모델 학습
    music_data = load_music_data(db_music)
    user_ids, uid_to_index, all_titles, matrix, user_likes = load_user_item_matrix(db_user)
    model = train_word2vec(music_data)

    genre_set = sorted({g.strip().lower() for song in music_data for g in (song.get("genres") or [])})
    genre_to_index = {g: i for i, g in enumerate(genre_set)}
    song_vectors = make_song_vectors(music_data, model, genre_to_index)

    # 추천 대상 설정
    target_user = "최민호"
    likes = user_likes[target_user]
    if len(likes) < 2:
        raise ValueError("유저 좋아요 곡 수가 너무 적습니다. 최소 2개 이상 필요.")

    ref_song = likes[0]
    ground_truth = likes[1:]  # 테스트용 실제 좋아요 곡

    # 추천 수행
    results = hybrid_recommendation(
        target_user,
        ref_song,
        user_ids,
        uid_to_index,
        matrix,
        user_likes,
        song_vectors,
        music_data,
        top_k=2,
        top_n=5
    )

    print(f"\n[최종 추천 결과 for '{target_user}']:")
    for title, artist, score in results:
        print(f"- {title} | {artist} | (점수: {score:.4f})")

    # 성능 평가
    evaluate_precision_recall_at_k(results, ground_truth, k=5)



[1] 음악 데이터를 불러옵니다...
   → 총 곡 수: 7689

[2] 유저 데이터를 불러옵니다...
   → 유저 수: 11, 곡 수: 157

[3] Word2Vec 모델 학습 중...
   → 학습된 단어 수: 264641

[4] 곡 벡터 생성 중...

[5] 하이브리드 추천 수행 중...
   → 사용자 '최민호'의 협업 유사도 계산:
     - 윤승서: 0.0333
     - 류희철: 0.1155
     - 서동진: 0.0000
     - 백나리: 0.0577
     - 규진: 0.0000
     - 김태호: 0.1291
     - 김태현: 0.0667
     - 이승은: 0.0000
     - 천지성: 0.0577
     - 김규표: 0.0000

   → 기준 곡: '청춘만화' (벡터 길이: 140)
   → '사랑하지마요' | CF: 0.1291, CBF: 0.5000, Hybrid: 0.3516
   → 'Rush (feat. 정인)' | CF: 0.1291, CBF: -0.0000, Hybrid: 0.0516
   → '동이 틀 때' | CF: 0.1291, CBF: 0.3137, Hybrid: 0.2398
   → 'TONIGHT' | CF: 0.1291, CBF: 0.7071, Hybrid: 0.4759
   → 'Cafe' | CF: 0.1291, CBF: 0.7071, Hybrid: 0.4759
   → '그래서 그래 (feat. 윤하)' | CF: 0.1291, CBF: 0.0000, Hybrid: 0.0517
   → '나는 반딧불' | CF: 0.1155, CBF: 0.0000, Hybrid: 0.0462
   → '뜨래요' | CF: 0.1155, CBF: 0.0000, Hybrid: 0.0462
   → '고독하구만' | CF: 0.1155, CBF: 0.0000, Hybrid: 0.0462
   → '봄 내음보다 너를' | CF: 0.1155, CBF: 0.0000, Hybrid: 0.0462
  